# Validación cruzada

Hasta ahora hemos aprendido la _validacion de entrenamiento/prueba_. Ahora nos concentraremos en técnicas más robustas. <br>

### Validacion de retención (_Holdout validation_).

La cual consiste en:
- Dividir el conjunto de datos completo en 2 particiones:
    - Un conjunto de entrenamiento.
    - Un conjunto de pruebas
    - Entrenar el modelo con el conjunto de entrenamiento.
- Utilizar el modelo entrenado para predecir etiquetas en el conjunto de pruebas.
- Calcular una métrica de error para comprender la eficacia del modelo.
- Cambiar los conjuntos de entrenamiento y de prueba y repetirlo.
- Promediar errores.

Nota: Con la validación _**holdout validation**_, se suele utilizar una division de 50/50 en lugar de 75/25 de la validación de entrenamiento/prueba. 

In [8]:
import numpy as np
import pandas as pd

# Leyendo datos
dc_listings = pd.read_csv('./dc_airbnb.csv')

# Limpiando y preparando columna de interes.
stripped_commas = dc_listings['price'].str.replace(',','')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')

# Ordenando aleatoriamente los datos
print(dc_listings.index)
shuffled_index = np.random.permutation(dc_listings.index)
dc_listings = dc_listings.reindex(shuffled_index)
dc_listings.head()

RangeIndex(start=0, stop=3723, step=1)


,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state
2980,100%,100%,1,3,Entire home/apt,1.0,1.0,1.0,90.0,$10.00,NaN,1,1125,11,38.936036,-77.036547,Washington,20010,DC
3241,100%,100%,3,6,Entire home/apt,3.0,1.0,3.0,130.0,$90.00,$400.00,2,365,41,38.920781,-76.994298,Washington,20018,DC
2849,85%,88%,1,2,Entire home/apt,1.0,1.0,1.0,90.0,NaN,NaN,2,1125,12,38.924530,-77.028121,Washington,20009,DC
1994,50%,NaN,1,4,Entire home/apt,1.0,1.0,3.0,110.0,NaN,$100.00,2,14,7,38.900107,-76.999835,Washington,20002,DC
2179,100%,100%,3,3,Private room,1.0,1.5,1.0,69.0,$12.00,$95.00,1,1125,5,38.905911,-77.007068,Washington,20001,DC


In [9]:
#################################################
# Dividiendo los datos para la validacion cruzada
#################################################

split_one = dc_listings.iloc[0:1862].copy()
split_two = dc_listings.iloc[1862:].copy()

In [10]:
#################################################
# Entrenando modelos 
#################################################

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

# Asignando datos de entrenamiento y prueba
train_one = split_one
test_one = split_two
train_two = split_two
test_two = split_one

# Creacion de modelo KNN, ajuste y prediccion para train y test one.
knn = KNeighborsRegressor(n_neighbors=5, algorithm='auto')
knn.fit(train_one[ ['accommodates'] ], train_one['price'])
test_one['predicted_price'] = knn.predict( test_one[ ['accommodates'] ] )
iteration_one_rmse = mean_squared_error(test_one['price'], test_one['predicted_price']) ** (1/2)

# Creacion de modelo KNN, ajuste y prediccion para train y test two
knn = KNeighborsRegressor(n_neighbors=5, algorithm='auto')
knn.fit(train_two[ ['accommodates'] ], train_two['price'])
test_two['predicted_price'] = knn.predict( test_two[[ 'accommodates' ]] )
iteration_two_rmse = mean_squared_error(test_two['price'], test_two['predicted_price']) ** (1/2)

In [11]:
#####################################
# Calculando el promedio de los RMSE
#####################################

avg_rmse = np.mean([iteration_one_rmse, iteration_two_rmse])

print(f'RMSE_one = {iteration_one_rmse}\nRMSE_two = {iteration_two_rmse}\nAVG = {avg_rmse}')

RMSE_one = 131.24339084713256
RMSE_two = 131.07984480569738
AVG = 131.16161782641495


### Validacion cruzada K-Fold

La _validacion holdout_ es en realidad un ejemplo específico de una clase mas amplia de técnicas de validacion llamada _**validación cruzada k-fold (K-fold cross-validation)**_. Esta validacion aprovecah una mayor proporcion de los datos durante el entrenamiento mientras sigue rotando a través de diferentes subconjuntos de los datos. <br>

La cual consiste en:
- Dividir el conjunto de datos en **K** particiones de igual longitud.
    - Seleccionar **K-1** particiones como conjunto de entrenamiento.
    - Seleccionar la partción restante como conjunto de prueba.
- Entrenamiento del modelo con el conjunto de entrenamiento.
- Utilizar el modelo entrenaod para predecir etiquetas en el conjunto de prueba.
- Calcular la metrica de error del conjunto de prueba.
- Repetir todos los pasos anteriores **K-1** veces, hasta que cada partición se haya utilizado como conjunto de prueba para una iteración.
- Calcular la media de los **K** valores de error.  